In [1]:
import numpy as np
np.seterr(all='raise')
import sys
sys.path.append('../')

In [2]:
from meshmaker.base import Base, Laziness
from meshmaker.model import Model
from meshmaker.mesh import Mesh
from meshmaker.tform import TForm
from meshmaker.vec3 import vec3
from meshmaker.quat import quat
from meshmaker.geometry import batch, slide, loop_offset, loop_normal, isnear, near, loopO
from collections import defaultdict

In [3]:
import os
import numpy as np
from pyrr import Matrix44
from PIL import Image
import moderngl
import moderngl_window
from moderngl_window.conf import settings
from moderngl_window.timers.clock import Timer
from meshmaker.base import Base, Laziness


class LazyMaterials(Laziness):
    
    def __init__(self, ctx, texture_directory, **kws):
        self.textures = {}
        for r, ds, fs in os.walk(texture_directory):
            for f in fs:
                name = f[:f.rfind('.')]
                path = os.path.join(r, f)
                self.textures[name] = path
        super().__init__(self.method(ctx), **kws)
    
    def __iter__(self):
        yield from self.textures.keys()
    
    def method(self, ctx):
        def wrapped(name):
            path = self.textures[name]
            img = Image.open(path)
            imgbytes = img.tobytes()
            bytesize = int(len(imgbytes) / np.product(img.size))
            texture = ctx.texture(img.size, bytesize, imgbytes)
            print(f'Loaded texture: {name} ({path})')
            return texture
        return wrapped

                
class Camera(Base):
    
    def mouse_drag_event(self, x, y, dx, dy):
        """Pan/rotate using left/right click/drag"""
        if self._last_mouse == 1:
            look = (self.target - vec3(*self.xyz)).nrm()
            left = vec3.Z().crs(look)
            up = look.crs(left)            
            self.target += (left * dx + up * dy) * self.speed
        elif self._last_mouse == 2:
            self.theta -= dx * self.speed
            self.phi -= dy * self.speed
        
    def mouse_scroll_event(self, x_offset, y_offset):
        self.radius -= y_offset * self.speed * 10
        
    def mouse_press_event(self, x, y, button):
        self._last_mouse = button
    
    def reset_view(self):
        self.target   = vec3(0, 0, 0)
        self.radius = 10
        self.theta  = np.pi / 3
        self.phi    = np.pi / 3
        self.aspect_ratio = 16 / 9
        self.speed = 0.01

    def __init__(self, prog):
        self.mvp = prog['Mvp']
        self.light = prog['Light']
        self.reset_view()
    
    @property
    def xyz(self):
        x = self.radius * np.cos(self.theta) * np.sin(self.phi)
        y = self.radius * np.sin(self.theta) * np.sin(self.phi)
        z = self.radius * np.cos(self.phi)
        return x + self.target.x, y + self.target.y, z + self.target.z
        
    def __call__(self):
        x, y, z = self.xyz
        proj = Matrix44.perspective_projection(
            45.0, self.aspect_ratio, 0.1, 100.0)
        lookat = Matrix44.look_at(
            (            x,             y,             z),
            (self.target.x, self.target.y, self.target.z),
            (            0,             0,             1))
        self.mvp.write((proj * lookat).astype('f4').tobytes())
        self.light.value = (x, y, z)


class NewWindow(Base):

    vertex_shader = '''
        #version 330

        uniform mat4 Mvp;

        in vec3 in_position;
        in vec3 in_normal;
        in vec2 in_texcoord_0;

        out vec3 v_vert;
        out vec3 v_norm;
        out vec2 v_text;

        void main() {
            gl_Position = Mvp * vec4(in_position, 1.0);
            v_vert = in_position;
            v_norm = in_normal;
            v_text = in_texcoord_0;
        }
    '''
    fragment_shader = '''
        #version 330

        uniform vec3 Light;
        uniform sampler2D Texture;

        in vec3 v_vert;
        in vec3 v_norm;
        in vec2 v_text;
        
        out vec4 f_color;
        
        void main() {
            float lum = clamp(dot(normalize(Light - v_vert), normalize(v_norm)), 0.0, 1.0) * 0.3 + 0.4;
            vec3 base = vec3(0.5, 0.5, 0.5) * lum;
            vec3 spec = vec3(1.0, 1.0, 1.0) * pow(lum, 5.7);
            vec4 tex = texture(Texture, v_text);
            f_color = vec4(base * 0.1 + tex.rgb * lum + spec, tex.a);
        }
    '''
    
    def __init__(self, source):
        self.source = source
        self.targets = []
        
        settings.WINDOW['class'] = 'moderngl_window.context.pyglet.Window'
        self.wnd = moderngl_window.create_window_from_settings()
        self.wnd.key_event_func = self.key_event
        
        self.ctx = self.wnd.ctx
        self.ctx.enable(moderngl.DEPTH_TEST | moderngl.CULL_FACE)

        self.prog = self.ctx.program(
            vertex_shader=self.vertex_shader,
            fragment_shader=self.fragment_shader)
        
        self.materials = LazyMaterials(self.ctx, '../resources/textures/')

        self.camera = Camera(self.prog)
        self.wnd.mouse_drag_event_func = self.camera.mouse_drag_event
        self.wnd.mouse_scroll_event_func = self.camera.mouse_scroll_event
        self.wnd.mouse_press_event_func = self.camera.mouse_press_event

        self.run()
    
    def key_event(self, key, action, modifiers):
        keys = self.wnd.keys
        if action == keys.ACTION_PRESS:
            if key == keys.Q:
                self.wnd.close()
            elif key == keys.U:
                self.update()
            elif key == keys.R:
                self.camera.reset_view()
            elif key == keys.W:
                self.ctx.wireframe = not self.ctx.wireframe
            else:
                parameters = getattr(self.source, 'parameters', {})
                operation = parameters.get(key)
                if operation is not None:
                    operation(key, action, modifiers)
                    self.update()

    def run(self):
        timer = Timer()
        timer.start()
        self.update()
        while not self.wnd.is_closing:
            self.wnd.clear()
            time, frame_time = timer.next_frame()
            self.render(time, frame_time)
            self.wnd.swap_buffers()
        self.wnd.destroy()

    def render(self, time, frame_time):
        self.ctx.clear(0.8, 0.8, 1.0)
        self.ctx.enable(moderngl.DEPTH_TEST)
        self.camera()
        for mode, texture, vao in self.targets:
            if texture is not None:
                texture.use()
            vao.render()
            vao.render(mode)

    def draw_material(self, tf, material, vertices=None, wireframe=None):
        """Recursively aggregate data from a scene graph for a material"""
        vertices = [] if vertices is None else vertices
        wireframe = [] if wireframe is None else wireframe
        if hasattr(tf, 'models'):
            for model in tf.models:
                for mesh in model.meshes.get(material, ()):
                    try:
                        tridata, wiredata = mesh.T2F_N3F_V3F(tf)
                        vertices.extend(tridata)
                        wireframe.extend(wiredata)
                    except:
                        print(f'failed to generate mesh {mesh}')
        for child in tf.children:
            self.draw_material(child, material, vertices, wireframe)
        return vertices, wireframe
        
    def update(self):
        """Recompute the scenegraph and vao data"""
        if hasattr(self.source, 'scene'):
            self.scene = self.source.scene()
        else:
            self.scene = TForm()
            print(f'Source has no "scene" method: {self.source}')

        self.targets = []
        for name in self.materials:
            vertices, wireframe = self.draw_material(self.scene, name)
            if vertices:
                vertices = np.array(vertices)
                vbo = self.ctx.buffer(vertices.astype('f4').tobytes())
                vao = self.ctx.simple_vertex_array(self.prog, vbo,
                    'in_texcoord_0', 'in_normal', 'in_position')
                self.targets.append((moderngl.TRIANGLES, self.materials[name], vao))
            
            
            if True and wireframe:
                #wireframe = np.array(wireframe)
                #vbo = self.ctx.buffer(wireframe.astype('f4').tobytes())
                axes = np.array([[[0, 0, 0], [1, 0, 0], [0, 0, 0]],
                                 [[0, 0, 0], [0, 1, 0], [0, 0, 0]],
                                 [[0, 0, 0], [0, 0, 1], [0, 0, 0]]])
                vbo = self.ctx.buffer(axes.astype('f4').tobytes())
                vao = self.ctx.simple_vertex_array(self.prog, vbo,
                    'in_position')
                self.targets.append((moderngl.LINES, None, vao))

    def axes(self, o=vec3.O()):
        raise
        self.lines['r'].append((o, o + vec3.X()))
        self.lines['g'].append((o, o + vec3.Y()))
        self.lines['b'].append((o, o + vec3.Z()))


def test():
    NewWindow(MetaMesh(make_control_mesh()))
    
test()

NameError: name 'MetaMesh' is not defined

In [4]:
class SimpleMesh(ParamMesh):
    
    @staticmethod
    def vertexfeature(control, vertices):
        positions = [control.vertices[v] for v in vertices]
        models = [cube_model(0.2, 'orangeboxtex')]
        #tfs = [TForm(position=p, models=models) for p in positions]
        tfs = [TForm(t=p, models=models) for p in positions]
        return TForm(children=tfs)

    @staticmethod
    def edgefeature(control, edges):
        edges = [(control.vertices[i], control.vertices[j]) for i, j in edges]
        positions = [u.lerp(v, 0.3) for u, v in edges]
        models = [cube_model(0.1, 'orangeboxtex')]
        #tfs = [TForm(position=p, models=models) for p in positions]
        tfs = [TForm(t=p, models=models) for p in positions]
        return TForm(children=tfs)
    
    @staticmethod
    def facefeature(control, faces):
        mesh = Mesh()
        for f in faces:
            mesh.af([control.vertices[v] for v in control.faces[f]])
        return TForm(models=[Model(meshes={'orangeboxtex': [mesh]})])

    def __init__(self, control, **kws):
        super().__init__(control, **kws)
        points = (self.vertexfeature, ())
        segments = (self.edgefeature, ())
        surfaces = [f for f, face in control]
        surfaces = (self.facefeature, surfaces)
        self.features = [points, segments, surfaces]

NameError: name 'ParamMesh' is not defined

In [5]:
def cube_model(r=1, material='dummyimage2'):
    mesh = cube_mesh(r=r)
    model = Model()
    model.add(material, mesh)
    return model


class ParamMesh(Base):
    """Proxy class for representing hierarchical
    mesh parameterizations via control meshes"""
    
    def __init__(self, control, **kws):
        super().__init__(features=[], **kws)
        self.control = control
        self.showcontrol = False
        
        from pyglet.window import key
        self.parameters = {
            key.C: self.on_toggle,
            key.UP: self.on_up, key.DOWN: self.on_down,
            key.LEFT: self.on_left, key.RIGHT: self.on_right,
            key.J: self.on_j, key.K: self.on_k,
        }

    def on_toggle(self, *ags):
        self.showcontrol = not self.showcontrol
    def on_left(self, *ags):
        vec3(0.9, 1, 1).sclps(self.control.vertices)
    def on_right(self, *ags):
        vec3(1.1, 1, 1).sclps(self.control.vertices)
    def on_up(self, *ags):
        vec3(1, 1.1, 1).sclps(self.control.vertices)
    def on_down(self, *ags):
        vec3(1, 0.9, 1).sclps(self.control.vertices)
    def on_j(self, *ags):
        vec3(1, 1, 0.9).sclps(self.control.vertices)
    def on_k(self, *ags):
        vec3(1, 1, 1.1).sclps(self.control.vertices)
        
    def scene(self):
        if self.showcontrol:
            parts = [TForm(models=[Model(meshes={'generic_8': [self.control]})])]
        else:
            parts = [f(self.control, target) for f, target in self.features]
        return TForm(children=parts)

    
class MetaMesh(ParamMesh):
    """A ParamMesh where features are inferred from meta data associated
    with the simplices of the control mesh"""

    @staticmethod
    def textured(texture):
        def facefeature(control, faces):
            mesh = Mesh()
            for f in faces:
                mesh.af([control.vertices[v] for v in control.faces[f]])
            return TForm(models=[Model(meshes={texture: [mesh]})])
        facefeature.__name__ = texture
        return facefeature

    def __init__(self, control, **kws):
        super().__init__(control, **kws)
        features = defaultdict(list)
        methods = {}
        for f, face in control:
            metas = control.meta.get(f)
            if not isinstance(metas, tuple):
                metas = (metas, )
            for meta in metas:
                if callable(meta):
                    methods[meta.__name__] = meta
                    features[meta.__name__].append(f)
                else:
                    print('whats goin on here', meta)
                    raise
        self.features = [(methods[m], faces) for m, faces in features.items()]

In [6]:
class Roof(Base):
    
    __name__ = 'FuckingRoof'
    
    def __init__(self, texture='generic_8', **kws):
        super().__init__(**kws)
        self.texture = texture
        
    def __call__(self, control, faces):
        mesh = Mesh()
        
        seams = control.perimeter(faces)
        
        control._face_normals = None
        normals = control.face_normals()
        cliffs = []
        for seam in seams:
            cliff = []
            for i, j in seam:
                inside  = control.e2f.get((i, j))
                outside = control.e2f.get((j, i))
                if near(normals[inside].z - normals[outside].z, 0) > 0:
                    inside = [control.vertices[v] for v in control.faces[inside]]
                    outside = [control.vertices[v] for v in control.faces[outside]]
                    inside_z = vec3.com(inside).z
                    outside_z = vec3.com(outside).z
                    if inside_z > outside_z:
                        cliff.append((i, j))
            else:
                if cliff:
                    cliffs.append(cliff)

        rims = [self.cliffrailing(control, cliff) for cliff in cliffs]
        
        for f in faces:
            mesh.af([control.vertices[v] for v in control.faces[f]])
        return TForm(models=[Model(meshes={self.texture: [mesh]})], children=rims)

    def cliffrailing(self, control, cliff, w=0.02, h=0.2):
        complete = cliff[0][0] == cliff[-1][1]
        loop = [control.vertices[i] for i, j in cliff]
        if not complete:
            loop.append(control.vertices[cliff[-1][1]])

        loops = [loop]
        loops.append([p.cp().ztrn( h) for p in loops[-1]])
        loops.append(loop_offset(loops[-1], w, complete))
        loops.append([p.cp().ztrn(-h) for p in loops[-1]])
        
        mesh = Mesh()
        mesh.bridges(loops, m=1, n=(0 if complete else 1))
        return TForm(models=[Model(meshes={self.texture: [mesh]})])

In [7]:
def jamb(loop, jw=0.02, jd=0.04, texture='generic_10'):
    """Hide the edges of loop with a mesh"""
    N = loop_normal(loop)
    loops = []
    loops.append((N * (jd / 2)).trnps([p.cp() for p in loop]))
    loops.append(loop_offset(loops[-1], jw))
    loops.append((N * (-jd)).trnps([p.cp() for p in loops[-1]]))
    loops.append(loop_offset(loops[-1], -jw))
    mesh = Mesh()
    mesh.bridges(loops, n=0, m=1)
    return TForm(models=[Model(meshes={texture: [mesh]})])        

In [8]:
class Entry(Base):

    __name__ = 'AnalEntry'

    def __init__(self, **kws):
        super().__init__(**kws)
        self.textures = {'wall': 'generic_1', 'stairs': 'generic_8', 'jamb': 'generic_10'}
        
    def __call__(self, control, faces):
        return TForm(children=[self.door(control, f) for f in faces])
    
    def door(self, control, f):
        dw, dh, dz, dd = 0.3, 0.6, 0.02, 0.04
        
        loop = [control.vertices[v] for v in control.faces[f]]
        o = loopO(loop)
        w = loop[o + 1].d(loop[o])
        h = loop[o - 1].d(loop[o])
        dw = min(dw, 0.8 * w)
        dh = min(dh, 0.8 * h - dz)
        T = (loop[o + 1] - loop[o]).nrm()
        Z = (loop[o - 1] - loop[o]).nrm()
        x = loop[o].lerp(loop[o + 1], 0.5)
        a = x - T * (dw / 2) + Z * dz
        b = a + T * (dw)
        w, z = b + Z * dh, a + Z * dh

        N = control.face_normals()[f]
        loops = [loop[o + 1:] + loop[:o + 1], [b, w, z, a]]
        loops.append([p - N * dd for p in loops[-1]])
        
        mesh = Mesh()
        mesh.bridges(loops, m=1)
        #mesh.bridge(loop[o + 1:] + loop[:o + 1], [b, w, z, a])
        #mesh.af(loops[-1])
        
        stoop = self.access(x, control, control.faces[f][o], control.faces[f][o + 1])
        thejamb = jamb(loops[-1], texture=self.textures['jamb'])
        
        return TForm(models=[Model(meshes={self.textures['wall']: [mesh]})],
                     children=[stoop, thejamb])
        
    def access(self, anchor, control, i, j):
        """Find a sequence of faces with sufficient steepness along edgeloop ji"""
        edges = control.edgeloop(i, j)
        edges.pop(0)

        height = 0
        direction = vec3.Z().crs(control.vertices[i] - control.vertices[j]).nrm()
        for i, j in edges:
            f = control.e2f.get((i, j))
            N = control.face_normals()[f]
            zs = [control.vertices[v].z for v in control.faces[f]]
            height += max(zs) - min(zs)
            if N.z > 0:
                break
        
        slope = 1.0
        sw = 0.5
        loop = []
        loop.append(anchor + vec3.nZ(height) + direction * (height / slope))
        loop.append(anchor.cp())
        loop.append(anchor.cp().ztrn(-height))

        N = loop_normal(loop)
        l = [(p + N * (sw / 2)) for p in loop]
        r = [(p - N * (sw / 2)) for p in loop]
        
        mesh = Mesh()
        mesh.af(l)
        mesh.af(r[::-1])
        patch = mesh.bridge(r, l, m=2)
        mesh.uvs = mesh.unwrap_uvs(patch[0])
        
        return TForm(models=[Model(meshes={self.textures['stairs']: [mesh]})])

In [9]:
def chunkloop(loop, chunk_w=1.01, chunk_h=1.01, n=None, m=None):
    o = loopO(loop)
    loop = loop[o:] + loop[:o]    
    lw = round(loop[0].d(loop[ 1]), 1)
    lh = round(loop[0].d(loop[-1]), 1)
    n = (int(lw / chunk_w) - 1) if n is None else n
    m = (int(lh / chunk_h) - 1) if m is None else m
    bottom = ([loop[0]] + loop[0].line(loop[1], n) + [loop[1]])
    top    = ([loop[3]] + loop[3].line(loop[2], n) + [loop[2]])
    lines  = [([x] + x.line(y, m) + [y]) for x, y in zip(bottom, top)]
    chunks = []
    for l, r in slide(lines, 2, 1):
        for (a, b), (c, d) in zip(slide(l, 2, 1), slide(r, 2, 1)):
            chunk = [a, c, d, b]
            chunks.append(chunk)
    return chunks, lw, lh

def test():
    from meshmaker.plt import plot, plot_loop
    
    l = vec3.O().ring(0.2, 4, False)
    
    f, ax = plot()
    plot_loop(ax, l, col='r', lw=2, mk='*')
    for l in chunkloop(l)[0]:
        plot_loop(ax, loop_offset(l, 0.1), col='b', lw=1, mk='*')

#test()

In [10]:
def trim(control, edges):
    edges = [(control.vertices[i], control.vertices[j]) for i, j in edges]
    positions = [u.lerp(v, 0.3) for u, v in edges]
    models = [cube_model(0.1, 'generic_9')]
    tfs = [TForm(position=p, models=models) for p in positions]
    return TForm(children=tfs)


class Wall(Base):
    
    __name__ = 'FuckingWall'
    
    def __call__(self, control, faces):
        self.ww = 0.3
        self.wh = 0.6
        self.wz = 0.2
        self.wd = 0.03
        self.wf = WindowFactory(self.ww, self.wh, self.wz, self.wd)

        mesh = Mesh()
        uv_seams = []
        children = []
        for f in faces:
            N = control.face_normals()[f]
            loop = [control.vertices[v] for v in control.faces[f]]
            chunks, w, h = chunkloop(loop)
            if w > self.ww * 1.1 and h > (self.wh + self.wz) * 1.1:
                windows = self.wf(chunks)
                for chunk, window in zip(chunks, windows.children):
                    seam = window.transform([p.cp() for p in windows.seam])
                    hole = (N * self.wd).trnps([p.cp() for p in seam]) # project seam into plane of chunk?                    
                    mesh.bridges([chunk, hole, seam], n=0, m=1)
                    a, b, c, d = (mesh._fp(hole[0]), mesh._fp(hole[1]),
                                  mesh._fp(hole[2]), mesh._fp(hole[3]))
                    uv_seams.append((a, b))
                    uv_seams.append((b, a))
                    uv_seams.append((c, d))
                    uv_seams.append((d, c))
                    children.append(windows)
            else:
                for chunk in chunks:
                    mesh.af(chunk)
        mesh.uvs = mesh.unwrap_uvs(Y=vec3.Z(), seams=uv_seams)
        return TForm(models=[Model(meshes={'generic_13': [mesh]})], children=children)

In [11]:
class WindowFactory(Base):
    
    __name__ = 'wfa'
    
    def __init__(self, ww, wh, wz, wd, **kws):
        loop = [vec3(-ww / 2, wd, wz),
                vec3( ww / 2, wd, wz),
                vec3( ww / 2, wd, wz + wh),
                vec3(-ww / 2, wd, wz + wh)]
        self.wz = wz
        self.seam = loop
        self.jamb = self.make_jamb(loop)
        
    def make_jamb(self, loop, jw=0.02, jd=0.04, texture='generic_10'):
        """Hide the edges of loop with a mesh"""
        N = loop_normal(loop)
        loops = []
        loops.append((N * (jd / 2)).trnps([p.cp() for p in loop]))
        loops.append(loop_offset(loops[-1], jw))
        loops.append((N * (-jd)).trnps([p.cp() for p in loops[-1]]))
        loops.append(loop_offset(loops[-1], -jw))
        loops = [list(loop) for loop in zip(*loops)]
        loops.reverse()
        mesh = Mesh()
        patches = mesh.bridges(loops, n=1, m=0)
        mesh.uvs = {}
        for patch in patches:
            mesh.uvs = mesh.unwrap_uvs(patch[1], X=vec3.nY(),
                seams=mesh.perimeter(patch), uvs=mesh.uvs)
        return Model(meshes={texture: [mesh]})

    def __call__(self, loops):
        tfs = []
        for loop in loops:
            p = loop[0].lerp(loop[1], 0.5)
            N = loop_normal(loop)
            if isnear(N.y, -1):
                q = None
            elif isnear(N.y, 1):
                q = quat.av(-np.pi, vec3.Z())
            else:
                q = quat.uu(vec3.nY(), N)
            tfs.append(TForm(t=p, q=q, models=[self.jamb]))
        return TForm(children=tfs, seam=self.seam)

In [13]:
def make_control_mesh():
    Z = vec3.Z()
    
    mesh = Mesh()
    
    grass = MetaMesh.textured('generic_0')
    tile = MetaMesh.textured('generic_1')
    foundation = MetaMesh.textured('generic_2')

    wall = Wall(m=0)
    roof = Roof()
    entry = Entry()
    
    r = 2.5
    bottom = [vec3(-r,-r, 0), vec3( r,-r, 0), vec3( r, r, 0), vec3(-r, r, 0)]
    mesh.grid(*bottom, 4, 4, meta=grass)
    
    footprint = []
    for f, face in mesh:
        if all((mesh.e2f.get((j, i)) is not None) for i, j in slide(face, 2)):
            footprint.append(f)
            mesh.meta[f] = foundation
    
    #footprint.pop(-1)
    patches = mesh.extrude(footprint, Z * 0.2)
    for f in patches[0]:
        mesh.meta[f] = roof
            
    patches = mesh.extrude(patches[0], Z)
    for f in patches[1]:
        mesh.meta[f] = tile
    mesh.meta[patches[1][-5]] = entry
    
    patches[0].pop(-1)
    patches[0].pop(-2)
    patches = mesh.extrude(patches[0], Z)
    for f in patches[1]:
        mesh.meta[f] = wall
    
    patches[0].pop(-2)
    patches = mesh.extrude(patches[0], Z)
    for f in patches[1]:
        mesh.meta[f] = wall

    patches = mesh.extrude(patches[0], Z)
    for f in patches[1]:
        mesh.meta[f] = wall
        
    mesh.uvs = mesh.unwrap_uvs()

    return mesh
    

def test():
    NewWindow(MetaMesh(make_control_mesh()))
    
test()

Loaded texture: generic_1 (../resources/textures/generics/generic_1.png)
Loaded texture: generic_13 (../resources/textures/generics/generic_13.png)
Loaded texture: generic_0 (../resources/textures/generics/generic_0.png)
Loaded texture: generic_2 (../resources/textures/generics/generic_2.png)
Loaded texture: generic_10 (../resources/textures/generics/generic_10.png)
Loaded texture: generic_8 (../resources/textures/generics/generic_8.png)


In [14]:
%load_ext line_profiler

In [ ]:
%lprun -T lprof0 -f Wall.__call__ test()

In [15]:
class ColumnFactory(Base):
    
    @classmethod
    def meta(cls):
        """"""
        control = Mesh()
        #tile = MetaMesh.textured('tile2')
        tile = MetaMesh.textured('generic_9')
        #wood = MetaMesh.textured('wood2')
        wood = MetaMesh.textured('generic_2')

        r, h = 1, 1
        bottom = [vec3(-r,-r, 0), vec3( r,-r, 0), vec3( r, r, 0), vec3(-r, r, 0)]
        top = [p.cp().ztrn(h) for p in bottom]
        
        control.grid(*bottom, 2, 2, meta=(cls.simple, tile))
        control.bridge(top, bottom, meta=tile)
        return MetaMesh(control)
    
    @staticmethod
    def column(r=0.1, n=16, h=1, texture='generic_0'):
        b = vec3.Z(0).ring(r, n)
        t = vec3.Z(h).ring(r, n)
        mesh = Mesh()
        mesh.bridge(b, t, m=0, e=0.00001)
        mesh.normals = mesh.vertex_normals(smooth=True)
        mesh.uvs = mesh.unwrap_uvs(Y=vec3.Z(), S=vec3(1, 1, 0))
        return [Model(meshes={texture: [mesh]})]
    
    @classmethod
    def simple(cls, control, faces):
        models = cls.column()
        columns = []
        vertices = set([v for f in faces for v in control.faces[f]])
        for v in vertices:
            p = control.vertices[v].cp()
            columns.append(TForm(t=p, models=models))
        return TForm(children=columns)


NewWindow(ColumnFactory.meta())

Loaded texture: generic_0 (../resources/textures/generics/generic_0.png)
Loaded texture: generic_9 (../resources/textures/generics/generic_9.png)


In [26]:
def test(r=2, fh=0.5, lh=1.0):
    gen0 = MetaMesh.textured('generic_0')
    gen3 = MetaMesh.textured('generic_3')
    gen6 = MetaMesh.textured('generic_6')
    roof = Roof('generic_8')
    entry = Entry()
    
    
    control = Mesh()
    
    bottom = [vec3(-r,-r, 0), vec3( r,-r, 0), vec3( r, r, 0), vec3(-r, r, 0)]
    patch = control.grid(*bottom, 4, 4, meta=gen0)
    
    facade = patch[0] + patch[1]
    facade = control.extrude(facade, vec3.Z(fh), meta=gen3)
    
    facade = control.extrude(facade[0], vec3.Z(lh), meta=gen6)

    for f in facade[1]:
        N = control.face_normals()[f]
        if near(N.x, 0) > 0:
            control.meta[f] = entry
    
    for f in facade[0]:
        control.meta[f] = roof
    
    control.uvs = control.unwrap_uvs()
    
    NewWindow(MetaMesh(control))
    
test()

Loaded texture: generic_1 (../resources/textures/generics/generic_1.png)
Loaded texture: generic_0 (../resources/textures/generics/generic_0.png)
Loaded texture: generic_6 (../resources/textures/generics/generic_6.png)
Loaded texture: generic_10 (../resources/textures/generics/generic_10.png)
Loaded texture: generic_3 (../resources/textures/generics/generic_3.png)
Loaded texture: generic_8 (../resources/textures/generics/generic_8.png)
